In [1]:
import numpy as np
import cv2 as cv
import glob
import os

In [2]:
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

In [3]:
image_files = glob.glob("calib_imgs/*.png")

In [4]:
len(image_files)

99

In [21]:
obj_point = np.zeros((8*5, 3), np.float32)
grid = np.mgrid[0:8,0:5]
obj_point[:,:2] = grid.T.reshape(-1,2)


In [5]:
chessboard_size = (8,5)
square_size = 3.0

In [9]:
obj_point = np.zeros((chessboard_size[0] * chessboard_size[1], 3), np.float32)
obj_point[:, :2] = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]].T.reshape(-1, 2)
obj_point = obj_point * square_size 

In [10]:
obj_points = []
img_points = []

In [11]:
for filename in image_files:
    img = cv.imread(filename)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
 
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (8,5), None)
 
    # If found, add object points, image points (after refining them)
    if ret == True:
        obj_points.append(obj_point)
 
        #corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), None)
        img_points.append(corners)
 
        # Draw and display the corners
        cv.drawChessboardCorners(img, (8,5), corners, ret)
        cv.imshow('img', img)
        cv.waitKey(500)
cv.destroyAllWindows()

In [ ]:
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(obj_points, img_points, gray.shape[::-1], None, None)

In [ ]:
np.savetxt(fname="dist",X=dist)
np.savetxt(fname="mtx",X=mtx)

In [ ]:
for filename in image_files:
    img = cv.imread(filename)
    h,  w = img.shape[:2]
    newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))
    dst = cv.undistort(img, mtx, dist, None, newcameramtx)
 
    # crop the image
    x, y, w, h = roi
    dst = dst[y:y+h, x:x+w]
    cv.imwrite('calibresult/'+filename[16:], dst)
    